https://jasdeep06.github.io/posts/Understanding-LSTM-in-Tensorflow-MNIST/

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/", one_hot=True)

# import tensorflow as tf
# mnist = tf.keras.datasets.mnist
# (x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')

time_steps = 28   # unrolled through 28 time steps
num_units=128     # hidden LSTM units
n_input = 28      # rows of 28 pixels
lr = 1e-3
n_classes = 10    # 0-9
batch_size = 128

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# weights and biases of appropriate shape
out_weights = tf.Variable(tf.random_normal([num_units, n_classes]))
out_bias = tf.Variable(tf.random_normal([n_classes]))

# defining placeholders
# input image placeholder
x = tf.placeholder(tf.float32, [None, time_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# processing the input tensor from [batch_size, n_steps, n_input] to
# "time_steps" number of [batch_size, n_input]
input = tf.unstack(x, time_steps, 1)

In [4]:
# use one layer of LSTMCell to define our static_rnn network
lstm_layer = tf.keras.layers.LSTMCell(num_units, unit_forget_bias=1)
outputs, _ = tf.nn.static_rnn(lstm_layer, input, dtype=tf.float32)

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


In [5]:
prediction = tf.matmul(outputs[-1], out_weights) + out_bias

In [6]:
# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y))
# optimization
opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

# model evaluation
correct_prediciton = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
acc = tf.reduce_mean(tf.cast(correct_prediciton, tf.float32))

In [7]:
# init vars
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
it = 1
while it < 800:
    batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
    batch_x = batch_x.reshape((batch_size, time_steps, n_input))
    feed_dict = {x: batch_x, y: batch_y}
    sess.run(opt, feed_dict=feed_dict)

    if it % 99 == 0:
        acc_val = sess.run(acc, feed_dict=feed_dict)
        loss_val = sess.run(loss, feed_dict=feed_dict)

        print("For iter:", it)
        print("Acc:", acc_val)
        print("Loss", loss_val)
        print("-" * 15)
    it += 1

For iter: 99
Acc: 0.8359375
Loss 0.5216467
---------------
For iter: 198
Acc: 0.8984375
Loss 0.29674888
---------------
For iter: 297
Acc: 0.890625
Loss 0.29140443
---------------
For iter: 396
Acc: 0.9296875
Loss 0.24170826
---------------
For iter: 495
Acc: 0.9921875
Loss 0.055558804
---------------
For iter: 594
Acc: 0.9453125
Loss 0.14953184
---------------
For iter: 693
Acc: 0.96875
Loss 0.081543714
---------------
For iter: 792
Acc: 0.9609375
Loss 0.13352157
---------------


In [8]:
#calculating test accuracy
test_data = mnist.test.images[:].reshape((-1, time_steps, n_input))
test_label = mnist.test.labels[:]
print("Testing Accuracy:", sess.run(acc, feed_dict={x: test_data, y: test_label}))
sess.close()

Testing Accuracy: 0.9695
